# An example notebook in R

This is an example notebook in R to show how to use the `sagemaker-run-notebook` tool to run R-based notebooks.

See the [ReadMe file](README.md) in this directory for information on how to run this notebook.

The demonstration here is derived from an example in Winston Chang's __R Graphics Cookbook__ in the section "Adding Fitted Regression Lines."

We use `ggplot2` for plotting the dataset, so you'll need to build a container that installs that. Running the shell script `./build-container.sh` in this directory will invoke the CLI tool `run-notebook create-container` with the right arguments for this. The resulting container will be called `r-notebook-runner`.

In [ ]:
library(MASS)
library(ggplot2)

This example uses the `biopsy` dataset from the `MASS` library included with R. This is a 699 sample dataset of biopsy results from breast cancer patients. It can be used to try out various machine learning algorithms. Here we just want to visualize the relationship between an input column and the biopsy result.

In [ ]:
b <- biopsy

The dataset has two classes, "benign" and "malignant" and we want to get a feel for how predictive the input columns are. We simply create a numeric feature that's 1 for the malignant samples and 0 for the benign samples. This will allow us to plot a sigmoid that gives a feeling for the relationship between that column and the result.

In [ ]:
b$classn[b$class == "benign"] <- 0
b$classn[b$class == "malignant"] <- 1

The biopsy dataset has 9 measured attributes. These are named `V1` through `V9`, which is not very helpful, but if you get help with `?biopsy` you'll get information about each column. Each of the attributes is scored as an integer from 1 to 10.

We'll mark the following cell as the parameter cell. You can look at the sidebar with the wrench icon when you have the cell selected and you'll see that the cell has the `parameters` tag in it's metadata. That's how papermill knows where to substitute in the parameters passed when you run the notebook.

By default, we'll see how predictive column `V1` ("clump thickness") is, but you can set the `columns` parameter to any of the values `V1` to `V9` when you run the notebook to use that column instead.

In [ ]:
# This is the parameter cell. Override with "V1" to "V9" to pick one of the 9 input variables to graph.
column = "V1"

Now we use `ggplot` to visualize the relationship between the selected column and the biopsy result. There are two parts to this visualization:

1. Point clouds at each of the possible positions. We use jitter and transparency to give us a feeling for the density at each of the 20 possible positions for an (attribute, result) pair.
2. A smoothed line plotted with a logit and error to help us understand the predictive power of a linear model on this attribute alone.

In [ ]:
ggplot(b, aes(x=.data[[column]], y=classn)) +
  geom_point(position=position_jitter(width=0.3, height=0.06), alpha=0.2, shape=21, size=1.5) + 
  stat_smooth(method=glm, method.args = list(family = "binomial"), formula=y ~ x)